In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier, BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_validate
import re
import string
import os,glob
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
import keras
from nltk.corpus import stopwords

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
excel_file = r'/content/drive/MyDrive/Embeddings/Fully compiled dataset Sentence New.xlsx'
df = pd.read_excel(excel_file)
mapping = {'Plagiarized': 1, 'Unplagiarized': 0}
df['Type'] = df['Type'].replace(mapping)
df_type_0 = df[df['Type'] == 0]
df_type_1 = df[df['Type'] == 1]

# Shuffle both DataFrames
df_type_0 = df_type_0.sample(frac=1).reset_index(drop=True)
df_type_1 = df_type_1.sample(frac=1).reset_index(drop=True)

# Calculate the length of the longest DataFrame
max_length = max(len(df_type_0), len(df_type_1))

# Create a new DataFrame to hold the alternating rows
randomized_df = pd.DataFrame(columns=df.columns)

# Iterate through both DataFrames and append rows alternately until one of them runs out
for i in range(max_length):
    if i < len(df_type_0):
        randomized_df = randomized_df.append(df_type_0.iloc[i], ignore_index=True)
    if i < len(df_type_1):
        randomized_df = randomized_df.append(df_type_1.iloc[i], ignore_index=True)
mapping = {'Plagiarized': 1, 'Unplagiarized': 0}
randomized_df['Type'] = randomized_df['Type'].replace(mapping)
randomized_df.drop('ID', axis=1, inplace=True)
randomized_df

<ipython-input-3-5e724572395f>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  randomized_df = randomized_df.append(df_type_0.iloc[i], ignore_index=True)
<ipython-input-3-5e724572395f>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  randomized_df = randomized_df.append(df_type_1.iloc[i], ignore_index=True)


,Text,Type
0,ابتدائی تعلیم حاصل كرنے كے بعد آپ نے اپنے خالو...,0
1,کراچی کی عمارت وزیر مینشن میں رہائش پذیر جناح ...,1
2,اس بات کو بھی دھیان میں رکھنا چاہئے کہ طویل عر...,0
3,سوفٹ ویئرز کے آنے سے پہلے ان معاملات کوحل کرنے...,1
4,ہم اکثر سنتے رہتے ہیں کہ باقاعدگی سے ورزش دل ک...,0
...,...,...
2515,آج ہم اکیسویں صدی میں زندگی گذار رہے ہیں۔ کچھ...,1
2516,اللہ تعالیٰ فرماتا ہے (ترجمہ) ’’اور وہ دونوں (...,0
2517,ــ’’ماموں قاسم موسیٰ ‘‘نے آپ کا نام محمد علی ...,1
2518,سر سید احمد خان نے اپنے افکار و نظریات سے ہندو...,0


In [ ]:
# prompt: count the data in randomized_df,grouped by lable

grouped_data = randomized_df.groupby('Type').count()
print(grouped_data)


      Text
Type      
0     1260
1     1260


In [ ]:
randomized_df['Text'] = randomized_df['Text'].apply(lambda x: " ".join(x.lower() for x in x.split())) #convert text into lower case

randomized_df['Text'] = randomized_df['Text'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])    # Remove URLs

#dataset_df['Text'] = dataset_df['Text'].apply(lambda x: " ".join(x for x in x.split() if x[0]!="@"))

#dataset_df['Text'] = dataset_df['Text'].apply(lambda x: " ".join(x for x in x.split() if x[0]!="#"))

#dataset_df['Text'] = dataset_df['Text'].str.replace('\d+', '')  #Remove digits

randomized_df['Text'] = randomized_df['Text'].map(lambda x: re.sub(r'\W+', ' ', x)) # Remove Punctuation marks

#stop = stopwords.words('english')
#dataset_df['Text'] = dataset_df['Text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop)) #Remove stop words

print("\nDataset After Text Preprocessing:")
print("=================================================\n")
randomized_df.head(7)


Dataset After Text Preprocessing:



,Text,Type
0,ابتدائی تعلیم حاصل كرنے كے بعد آپ نے اپنے خالو...,0
1,کراچی کی عمارت وزیر مینشن میں رہائش پذیر جناح ...,1
2,اس بات کو بھی دھیان میں رکھنا چاہئے کہ طویل عر...,0
3,سوفٹ ویئرز کے آنے سے پہلے ان معاملات کوحل کرنے...,1
4,ہم اکثر سنتے رہتے ہیں کہ باقاعدگی سے ورزش دل ک...,0
5,یہ بات بھی مسل م ہے کہ ماں دنیامیں لانے کا سبب...,1
6,محمد علی جناح 1876 1948 ء قائد اعظم کے لقب سے ...,0


In [ ]:
!pip install --upgrade gensim -q
!pip install matplotlib -q

In [ ]:
import gensim
from gensim.models import word2vec
from gensim.models.word2vec import Word2Vec
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import spacy
import string

In [ ]:
gensim.__version__

'4.3.2'

In [ ]:
np.random.seed(42)

In [ ]:
import gensim.downloader as api
print(list(gensim.downloader.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [ ]:
#wv = api.load('word2vec-google-news-300')

In [ ]:
#wv.save('/content/drive/MyDrive/Embeddings/word2vec-google-news-300.kv')

In [ ]:
from gensim.models import KeyedVectors
wv = KeyedVectors.load('/content/drive/MyDrive/Embeddings/word2vec-google-news-300.kv')

In [ ]:
def sent_vec(sent):
    vector_size = wv.vector_size
    wv_res = np.zeros(vector_size)
    # print(wv_res)
    ctr = 1
    for w in sent:
        if w in wv:
            ctr += 1
            wv_res += wv[w]
    wv_res = wv_res/ctr
    return wv_res

In [ ]:
# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    doc = nlp(sentence)



    # print(doc)
    # print(type(doc))

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() for word in doc ]

    # print(mytokens)

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

In [ ]:
sent_vec("I am happy")

In [ ]:
randomized_df['vec'] = randomized_df['Text'].apply(sent_vec)

In [ ]:
randomized_df.head()

In [ ]:
X = randomized_df['vec'].to_list()
y = randomized_df['Type'].to_list()

In [ ]:
  X[0]

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier_LR = LogisticRegression()

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

# Assuming X and y are your feature matrix and target vector respectively


# Initialize StratifiedKFold with 5 folds
skf = StratifiedKFold(n_splits=5)



In [ ]:
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score, average='weighted'),
    'recall': make_scorer(recall_score, average='weighted'),
    'f1_score': make_scorer(f1_score, average='weighted')
}

In [ ]:
# Perform cross-validation
cv_results = cross_validate(classifier_LR, X, y, cv=skf, scoring=scoring)


In [ ]:
# Display the cross-validation scores
print("Mean Accuracy:", cv_results['test_accuracy'].mean())
print("Mean Precision:", cv_results['test_precision'].mean())
print("Mean Recall:", cv_results['test_recall'].mean())
print("Mean F1 Score:", cv_results['test_f1_score'].mean())

Mean Accuracy: 0.5904761904761905
Mean Precision: 0.5986079254317808
Mean Recall: 0.5904761904761905
Mean F1 Score: 0.5817535123094757


In [ ]:
dtree = DecisionTreeClassifier()

In [ ]:
# Perform cross-validation
cv_results = cross_validate(dtree, X, y, cv=skf, scoring=scoring)


In [ ]:
# Display the cross-validation scores
print("Mean Accuracy:", cv_results['test_accuracy'].mean())
print("Mean Precision:", cv_results['test_precision'].mean())
print("Mean Recall:", cv_results['test_recall'].mean())
print("Mean F1 Score:", cv_results['test_f1_score'].mean())

Mean Accuracy: 0.6408730158730159
Mean Precision: 0.6443602040551732
Mean Recall: 0.6408730158730159
Mean F1 Score: 0.6386685491047281


In [ ]:
# Initialize classifiers
logreg = LogisticRegression()
dtree = DecisionTreeClassifier()
rf = RandomForestClassifier()
svm = SVC()
knn = KNeighborsClassifier()
nb = GaussianNB()
lda = LinearDiscriminantAnalysis()
mlp = MLPClassifier(max_iter=1000)  # Adjust max_iter based on convergence
xgb = GradientBoostingClassifier()

In [ ]:
# Create a voting classifier
voting_clf = VotingClassifier(estimators=[
    ('logreg', logreg),
    ('dtree', dtree),
    ('rf', rf),
    ('svm', svm),
    ('knn', knn),
    ('nb', nb),
    ('lda', lda),
    ('mlp', mlp),
    ('xgb', xgb)
], voting='hard')

In [ ]:
# Initialize classifiers list
classifiers = [logreg, dtree, rf, svm, knn, nb, lda, mlp, xgb, voting_clf]

In [ ]:
from sklearn.model_selection import cross_val_score, KFold

In [ ]:
n_folds = 5

# Initialize KFold object
kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

In [ ]:
# Iterate over classifiers
for clf in classifiers:
    # Perform k-fold cross-validation
    scores = cross_val_score(clf, X, y, cv=kf, scoring='accuracy')

    # Calculate mean accuracy
    mean_accuracy = np.mean(scores)

    # Print mean accuracy
    print(f"Mean accuracy for {clf.__class__.__name__}: {mean_accuracy:.4f}")

Mean accuracy for LogisticRegression: 0.5933
Mean accuracy for DecisionTreeClassifier: 0.6437
Mean accuracy for RandomForestClassifier: 0.6500
Mean accuracy for SVC: 0.5956
Mean accuracy for KNeighborsClassifier: 0.6111
Mean accuracy for GaussianNB: 0.5270
Mean accuracy for LinearDiscriminantAnalysis: 0.6103
Mean accuracy for MLPClassifier: 0.5647
Mean accuracy for GradientBoostingClassifier: 0.6520
Mean accuracy for VotingClassifier: 0.6187


In [ ]:
# Iterate over classifiers
for clf in classifiers:
    # Perform k-fold cross-validation
    #scores = cross_val_score(clf, X, y, cv=kf, scoring='accuracy')

    # Calculate mean accuracy
    #mean_accuracy = np.mean(scores)

    # Print mean accuracy
    #print(f"Mean accuracy for {clf.__class__.__name__}: {mean_accuracy:.4f}")

    scores_accuracy = cross_val_score(clf, X, y, cv=skf, scoring='accuracy')
    scores_recall = cross_val_score(clf, X, y, cv=skf, scoring='recall')
    scores_precision = cross_val_score(clf, X, y, cv=skf, scoring='precision')
    scores_f1 = cross_val_score(clf, X, y, cv=skf, scoring='f1')

    # Calculate mean metrics
    mean_accuracy = np.mean(scores_accuracy)
    mean_recall = np.mean(scores_recall)
    mean_precision = np.mean(scores_precision)
    mean_f1 = np.mean(scores_f1)

    # Print metrics
    print(f"Metrics for {clf.__class__.__name__}:")
    print(f"**Accuracy:** {mean_accuracy:.4f}")
    print(f"**Recall:** {mean_recall:.4f}")
    print(f"**Precision:** {mean_precision:.4f}")
    print(f"**F1 Score:** {mean_f1:.4f}\n")


Metrics for LogisticRegression:
**Accuracy:** 0.5905
**Recall:** 0.4484
**Precision:** 0.6266
**F1 Score:** 0.5222

Metrics for DecisionTreeClassifier:
**Accuracy:** 0.6341
**Recall:** 0.7119
**Precision:** 0.6230
**F1 Score:** 0.6624

Metrics for RandomForestClassifier:
**Accuracy:** 0.6472
**Recall:** 0.7206
**Precision:** 0.6308
**F1 Score:** 0.6721

Metrics for SVC:
**Accuracy:** 0.5885
**Recall:** 0.3833
**Precision:** 0.6501
**F1 Score:** 0.4818

Metrics for KNeighborsClassifier:
**Accuracy:** 0.6060
**Recall:** 0.6238
**Precision:** 0.6024
**F1 Score:** 0.6088

Metrics for GaussianNB:
**Accuracy:** 0.5270
**Recall:** 0.1976
**Precision:** 0.5786
**F1 Score:** 0.2941

Metrics for LinearDiscriminantAnalysis:
**Accuracy:** 0.6099
**Recall:** 0.5063
**Precision:** 0.6392
**F1 Score:** 0.5645

Metrics for MLPClassifier:
**Accuracy:** 0.5948
**Recall:** 0.4024
**Precision:** 0.6213
**F1 Score:** 0.6135

Metrics for GradientBoostingClassifier:
**Accuracy:** 0.6500
**Recall:** 0.7310
**

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
# Define hyperparameter grids for each classifier
param_grid = {
    'LogisticRegression': {
        'C': [0.1, 1, 10, 100],
        'penalty': ['l1', 'l2']
    },
    'DecisionTreeClassifier': {
        'max_depth': [None, 5, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    },
    'RandomForestClassifier': {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 5, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    },
    'SVC': {
        'C': [0.1, 1, 10, 100],
        'gamma': [0.1, 1, 10]
    },
    'KNeighborsClassifier': {
        'n_neighbors': [3, 5, 10],
        'weights': ['uniform', 'distance']
    },
    'GaussianNB': {},

    'GradientBoostingClassifier': {
        'n_estimators': [100, 200, 300],
        'learning_rate': [0.01, 0.1, 0.5],
        'max_depth': [3, 5, 10],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    },
    'VotingClassifier': {}  # No hyperparameters to tune for VotingClassifier
}

In [ ]:
# Perform random search for each classifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

for clf in classifiers:
    clf_name = clf.__class__.__name__
    print(f"Optimizing hyperparameters for {clf_name}...")

    if clf_name in param_grid:
        param_dist = param_grid[clf_name]
        random_search = RandomizedSearchCV(estimator=clf, param_distributions=param_dist, n_iter=10, cv=5, scoring='accuracy', random_state=42)
        random_search.fit(X, y)

        print("Best parameters found:")
        print(random_search.best_params_)

        # Perform cross-validation with best parameters
        best_clf = random_search.best_estimator_
        scores_accuracy = cross_val_score(best_clf, X, y, cv=5, scoring='accuracy')
        scores_recall = cross_val_score(best_clf, X, y, cv=5, scoring='recall')
        scores_precision = cross_val_score(best_clf, X, y, cv=5, scoring='precision')
        scores_f1 = cross_val_score(best_clf, X, y, cv=5, scoring='f1')

        # Calculate mean metrics
        mean_accuracy = np.mean(scores_accuracy)
        mean_recall = np.mean(scores_recall)
        mean_precision = np.mean(scores_precision)
        mean_f1 = np.mean(scores_f1)

        # Print mean metrics
        print(f"Mean Accuracy: {mean_accuracy:.4f}")
        print(f"Mean Recall: {mean_recall:.4f}")
        print(f"Mean Precision: {mean_precision:.4f}")
        print(f"Mean F1 Score: {mean_f1:.4f}\n")
    else:
        print("No hyperparameters to tune.")


Optimizing hyperparameters for LogisticRegression...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 8 is smaller than n_iter=10. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown i

Best parameters found:
{'penalty': 'l2', 'C': 100}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Mean Accuracy: 0.6179
Mean Recall: 0.5468
Mean Precision: 0.6376
Mean F1 Score: 0.5884

Optimizing hyperparameters for DecisionTreeClassifier...
Best parameters found:
{'min_samples_split': 10, 'min_samples_leaf': 4, 'max_depth': 10}
Mean Accuracy: 0.6401
Mean Recall: 0.7087
Mean Precision: 0.6167
Mean F1 Score: 0.6622

Optimizing hyperparameters for RandomForestClassifier...
Best parameters found:
{'n_estimators': 300, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_depth': 10}
Mean Accuracy: 0.6496
Mean Recall: 0.7190
Mean Precision: 0.6301
Mean F1 Score: 0.6716

Optimizing hyperparameters for SVC...
Best parameters found:
{'gamma': 10, 'C': 100}
Mean Accuracy: 0.6385
Mean Recall: 0.7151
Mean Precision: 0.6203
Mean F1 Score: 0.6643

Optimizing hyperparameters for KNeighborsClassifier...


...

Best parameters found:
{'weights': 'distance', 'n_neighbors': 10}
Mean Accuracy: 0.6171
Mean Recall: 0.6341
Mean Precision: 0.6120
Mean F1 Score: 0.6217

Optimizing hyperparameters for GaussianNB...
Best parameters found:
{}


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Mean Accuracy: 0.5270
Mean Recall: 0.1976
Mean Precision: 0.5786
Mean F1 Score: 0.2941

Optimizing hyperparameters for LinearDiscriminantAnalysis...
No hyperparameters to tune.
Optimizing hyperparameters for MLPClassifier...
No hyperparameters to tune.
Optimizing hyperparameters for GradientBoostingClassifier...
Best parameters found:
{'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_depth': 10, 'learning_rate': 0.5}
Mean Accuracy: 0.6516
Mean Recall: 0.7278
Mean Precision: 0.6316
Mean F1 Score: 0.6759

Optimizing hyperparameters for VotingClassifier...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best parameters found:
{}
Mean Accuracy: 0.6222
Mean Recall: 0.5587
Mean Precision: 0.6475
Mean F1 Score: 0.5571

